In [84]:
import copy
import csv
import json
import numpy as np
import pandas as pd
import re

from collections import Counter
from io import StringIO

In [112]:
CENSUS_DATA_LOCATION = 'build/ACS_14_5YR_B19301_with_ann.csv'
ELECTION_DATA_LOCATION = 'python/data.csv'
OUTPUT_LOCATION = 'build/merged_data.csv'
US_JSON_LOCATION = 'us.json'
OUTPUT_US_JSON_LOCATION = 'public/data/us2016income.json'

df = pd.read_csv(ELECTION_DATA_LOCATION)

lines = []

with open(CENSUS_DATA_LOCATION, encoding='latin1') as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        count = line[3]
        count = re.match(r'(\d+)(?=\(r\d+\))?', count).groups()[0]
        lines.append([line[0], line[1], line[2], int(count)])

pop_df = pd.DataFrame.from_records(lines, columns=['id', 'id2', 'name', 'total'])

In [113]:
# Only used for juicing PC Income
lines = []
with open('build/ACS_14_5YR_B01003_with_ann.csv', encoding='latin1') as f:
    reader = csv.reader(f)
    next(reader)
    for line in reader:
        count = line[3]
        count = re.match(r'(\d+)(?=\(r\d+\))?', count).groups()[0]
        lines.append([line[0], line[1], line[2], int(count)])
odf = pd.DataFrame.from_records(lines, columns=['id', 'id2', 'name', 'total_pop'])
odf2 = pop_df.merge(odf, how='inner', on='id2')
odf2 = odf2.loc[:, ['id_x', 'id2', 'name_x', 'total', 'total_pop']].rename(columns={'name_x': 'name', 'id_x': 'id'})
odf2.loc[:, 'total'] = odf2['total'] * odf2['total_pop']
pop_df = odf2

In [114]:
df.head()

,state,county,reporting,dem,gop,lib,grn,una,oth
0,AL,Autauga,100%,5908,18110,538,105,0,0
1,AL,Baldwin,100%,18409,72780,2448,453,0,0
2,AL,Barbour,100%,4848,5431,93,18,0,0
3,AL,Bibb,100%,1874,6733,124,17,0,0
4,AL,Blount,100%,2150,22808,337,89,0,0


In [115]:
pop_df.head()

,id,id2,name,total,total_pop
0,0500000US01001,01001,"Autauga County, Alabama",1358771584,55136
1,0500000US01003,01003,"Baldwin County, Alabama",5134045455,191205
2,0500000US01005,01005,"Barbour County, Alabama",470514650,27119
3,0500000US01007,01007,"Bibb County, Alabama",410245830,22653
4,0500000US01009,01009,"Blount County, Alabama",1181780145,57645


In [116]:
pop_df['county'] = [x[0] for x in pop_df.name.str.split(',')]
pop_df['state'] = [x[1] for x in pop_df.name.str.split(',')]

In [117]:
STATES = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

STATE_ABBREVS = [
  'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',
  'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
  'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH',
  'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI',
  'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI',
  'WY']

STATE_TO_ABBREV = dict(zip(STATES, STATE_ABBREVS))

assert len(STATE_TO_ABBREV) == 51

In [118]:
pop_df['state'] = np.array(pop_df.name.apply(lambda x: x.split(',')[1]))
pop_df['state'] =pop_df['state'].str.strip()
pop_df['state_abbrev'] = pop_df['state'].apply(lambda x: STATE_TO_ABBREV[x])

In [119]:
# All found corrections
df.loc[df['county'] == 'Bedford Co.', 'county'] = 'Bedford'
pop_df.loc[pop_df['name'] == 'Doña Ana County, New Mexico', 'name'] = 'Dona Ana County, New Mexico'
df.loc[df['county'] == 'Sainte Genevieve', 'county'] = 'Ste. Genevieve'
df = df.loc[df['county'] != 'Kalawao']

In [120]:
# Election data in AK only at state level. Combine....
ak_population = pop_df[pop_df['state_abbrev'] == 'AK'].total.sum()
ak_df = pd.DataFrame.from_records([(-1, -1, 'Alaska, Alaska', ak_population, 'Alaska', 'Alaska', 'AK')],
                                  columns=('id', 'id2', 'name', 'total', 'county', 'state', 'state_abbrev'))
pop_df = pop_df.loc[~(pop_df['state_abbrev'] == 'AK')]
pop_df = pd.concat([pop_df, ak_df])

In [121]:
pop_df[pop_df['state_abbrev'] == 'AK']

,county,id,id2,name,state,state_abbrev,total,total_pop
0,Alaska,-1,-1,"Alaska, Alaska",Alaska,AK,24128028977,NaN


In [122]:
# Then there are a bunch of special cases.....
records_to_join_on = []

def row_with_county(row, county):
    records_to_join_on.append((row.id, row.id2, row.name, row.total,
                               county, row.state, row.state_abbrev))

# Fix Bedford County and City together (townhall only has results for one)    
bedford_total = pop_df[pop_df.county.apply(lambda x: 'Bedford' in x) & (pop_df.state_abbrev == 'VA')].total.sum()
pop_df = pop_df[~(pop_df.county.apply(lambda x: 'Bedford city' in x) & (pop_df.state_abbrev == 'VA'))]
pop_df.ix[pop_df.county.apply(lambda x: 'Bedford County' in x) & (pop_df.state_abbrev == 'VA'), 'total'] = bedford_total

for i, row in pop_df.iterrows():
    name = row['name']
    state = row.state_abbrev
    if name.startswith('Baltimore') and state == 'MD':
        if 'ity' in name:
            row_with_county(row, 'Baltimore City')
        else:
            row_with_county(row, 'Baltimore County')
    elif name.startswith('Carson City') and state == 'NV':
        row_with_county(row, 'Carson City')
    elif state == 'VA' and any(name.startswith(x) for x in ('Fairfax', 'Franklin', 'Richmond', 'Roanoke')):
        if 'County' in name:
            row_with_county(row, name.split(' ')[0] + ' Co.')
        else:
            row_with_county(row, name.split(' ')[0])
    elif name.startswith('LaSalle'):
        row_with_county(row, 'La Salle')
    elif state == 'DC':
        row_with_county(row, 'Washington')
    elif state == 'MO' and name.startswith('St. Louis'):
        if 'County' in name:
            row_with_county(row, 'St. Louis Co.')
        else:
            row_with_county(row, 'St. Louis')
    elif state == 'HI' and 'Kalawao' in name:
        # This is some uninhabited place
        pass
    elif state == 'SD' and 'Shannon' in name:
        # Name change in 2015
        row_with_county(row, 'Oglala Lakota')
    elif state == 'AK':
        row_with_county(row, 'Alaska')
    else:
        match = re.match(r'^(.*) (?=County|Parish|city)', name)
        if not match:
            print("WTF", row)
        else:
            row_with_county(row, match.groups()[0])

In [123]:
pop_df_fixed = pd.DataFrame.from_records(
    records_to_join_on, columns=['id', 'id2', 'name', 'total', 
                                 'county', 'state', 'state_abbrev'])

In [124]:
merged_df = df.merge(pop_df_fixed, how='inner', 
         left_on=['county', 'state'],
         right_on=['county', 'state_abbrev'])

assert len(merged_df) == len(df)

In [125]:
{(row.county, row.state_abbrev) for _, row in pop_df_fixed.iterrows()} - {(row.county, row.state) for _, row in df.iterrows()}

set()

In [126]:
merged_df.rename(columns={'state_y': 'full_state', 'state_x': 'state'}, inplace=True)
del merged_df['state_abbrev']

In [127]:
merged_df.to_csv(OUTPUT_LOCATION, index=False)

In [128]:
color = {}
for state in 'AK, AL, AR, CT, DE, HI, IL, ME, MI, MN, MT, NE, NM, NV, SC, VA, WA'.split(', '):
    color[state] = 0
for state in 'AZ, DC, FL, KS, KY, MS, NC, ND, OR, PA, RI, TX, VT, WI, WY'.split(', '):
    color[state] = 1
for state in 'CA, CO, GA, ID, IN, LA, MA, MO, NJ, SD, WV'.split(', '):
    color[state] = 2
for state in 'IA, MD, NH, NY, OH, OK, TN, UT'.split(', '):
    color[state] = 3

id_to_properties = {int(row.id2): {'state': row.state,
                                   'color': color[row.state],
                                   'name': row.county, 
                                   'population': row.total,
                                   'dem': row.dem if np.isfinite(row.dem) else 0,
                                   'gop': row.gop if np.isfinite(row.gop) else 0, 
                                   'grn': row.grn if np.isfinite(row.grn) else 0,
                                   'lib': row.lib if np.isfinite(row.lib) else 0,
                                   'una': row.una if np.isfinite(row.una) else 0,
                                   'oth': row.oth if np.isfinite(row.oth) else 0}
                   for _, row in merged_df.iterrows()}

In [129]:
with open(US_JSON_LOCATION, 'r') as f:
    data = json.load(f)
    
data['objects']['counties']['geometries'] = [x for x in data['objects']['counties']['geometries'] if x['id'] < 60000]
data['objects']['counties']['geometries'] = [x for x in data['objects']['counties']['geometries'] if x['id'] // 1000 != 2]

In [130]:
for val in data['objects']['counties']['geometries']:
    if val['id'] in id_to_properties:
        val['properties'] = id_to_properties[val['id']]

In [131]:
data['objects']['counties']['geometries'][0]

{'arcs': [[[0, 1, 2]]],
 'id': 53073,
 'properties': {'color': 0,
  'dem': 59402,
  'gop': 40247,
  'grn': 2954,
  'lib': 4745,
  'name': 'Whatcom',
  'oth': 875,
  'population': 5463687705,
  'state': 'WA',
  'una': 0},
 'type': 'MultiPolygon'}

In [132]:
# Fix AK
ak_geometry = copy.deepcopy([x for x in data['objects']['states']['geometries'] if x['id'] == 2][0])

In [133]:
ak_row = merged_df[merged_df.state == 'AK'].iloc[0]

ak_geometry['properties'] = {
    'color': 0,
    'dem': int(ak_row.dem),
    'gop': int(ak_row.gop),
    'grn': int(ak_row.grn),
    'lib': int(ak_row.lib),
    'una': int(ak_row.una),
    'name': 'Alaska',
    'oth': int(ak_row.oth),
    'population': int(ak_row.total),
    'state': 'AK'
}

ak_geometry['id'] = 2001

In [134]:
data['objects']['counties']['geometries'].append(ak_geometry)
data['objects']['counties']['geometries'].sort(key=lambda x: x['id'])

In [135]:
with open(OUTPUT_US_JSON_LOCATION, 'wt') as f:
    json.dump(data, f)